## 손익계산서 합치기

In [19]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
## 그냥 데이터프레임 가져올때 뜨는 경고창 무시하는 코드

In [20]:
## 정정 이전 재무제표

In [21]:
import pandas as pd

file_paths = [
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2015.11.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2016.05.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2016.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2017.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2017.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2018.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2018.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2019.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2019.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2020.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2020.11.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2021.05.17)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2021.11.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2022.05.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2022.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2023.05.15)_ko.xls"
]

dfs = []

for path in file_paths:
    # 기본 시도: header=[5,6]
    try:
        raw = pd.read_excel(path, sheet_name="포괄손익계산서", header=[5, 6])
    except Exception:
        # 만약 파일 구조가 다르면 header=[3,4]로 재시도
        raw = pd.read_excel(path, sheet_name="포괄손익계산서", header=[3, 4])

    # MultiIndex → 평탄화
    flat_cols = []
    for i, col in enumerate(raw.columns):
        if i == 0:
            flat_cols.append("항목")
        else:
            if isinstance(col, tuple):
                top, sub = col
                flat_cols.append(f"{str(top).strip()}_{str(sub).strip()}")
            else:
                flat_cols.append(str(col))
    raw.columns = flat_cols

    # 항목 전처리
    raw["항목"] = (
        raw["항목"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)
    )

    # "3개월" 컬럼만 추출
    three_cols = [c for c in raw.columns if "3개월" in c]

    # 못 찾았으면 header=[3,4]로 다시 읽기 시도 (혹시 skiprows 달라서 누락된 경우)
    if not three_cols:
        raw_alt = pd.read_excel(path, sheet_name="포괄손익계산서", header=[3, 4])
        flat_cols = []
        for i, col in enumerate(raw_alt.columns):
            if i == 0:
                flat_cols.append("항목")
            else:
                if isinstance(col, tuple):
                    top, sub = col
                    flat_cols.append(f"{str(top).strip()}_{str(sub).strip()}")
                else:
                    flat_cols.append(str(col))
        raw_alt.columns = flat_cols
        three_cols = [c for c in raw_alt.columns if "3개월" in c]
        if three_cols:
            raw = raw_alt  # 성공했으면 교체
        else:
            continue

    # 첫 번째 3개월 컬럼 사용
    three_col = three_cols[0]

    # 파일명에서 날짜 추출
    date_tag = path.split("재무제표(")[-1].split(")_ko.")[0]

    # 필요한 컬럼만 남기고 날짜로 이름 교체
    df = raw[["항목", three_col]].copy()
    df.rename(columns={three_col: date_tag}, inplace=True)
    dfs.append(df)

# 모든 DF 병합
if not dfs:
    raise ValueError("📛 어떤 파일에서도 3개월 컬럼을 찾지 못했습니다!")

df_merged = dfs[0]
for next_df in dfs[1:]:
    df_merged = pd.merge(df_merged, next_df, on="항목", how="outer")

df_merged = df_merged.sort_values("항목").reset_index(drop=True)

save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date1.xlsx"
df_merged.to_excel(save_path, index=False)

print("\n✅ 병합 완료! header=[5,6]→[3,4] 자동 전환 포함 →", save_path)




✅ 병합 완료! header=[5,6]→[3,4] 자동 전환 포함 → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date1.xlsx


In [22]:
## 정정 이후 재무제표 

In [23]:
import pandas as pd
import re

file_paths = [
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[(주)만도]3분기보고서_재무제표(2023.11.14)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]1분기보고서_재무제표(2024.05.16)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]3분기보고서_재무제표(2024.11.14)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]1분기보고서_재무제표(2025.05.15)_ko.xlsx"
]

dfs = []

for path in file_paths:
    df = pd.read_excel(path, sheet_name="D431415", skiprows=4)

    # 1️⃣ 첫 번째 컬럼명을 '항목'으로
    df.rename(columns={df.columns[0]: "항목"}, inplace=True)

    # 2️⃣ 항목 전처리
    df["항목"] = (
        df["항목"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)
    )

    # 3️⃣ 파일명에서 기준일(날짜) 추출 → 예: 2023.11.14
    date_tag = path.split("재무제표(")[-1].split(")_ko.xlsx")[0]

    # 4️⃣ 첫 번째 값 컬럼만 선택
    first_value_col = df.columns[1]
    sub = df[["항목", first_value_col]].copy()

    # 5️⃣ 열 이름 정리 — 파일명 날짜만 남김
    sub.rename(columns={first_value_col: date_tag}, inplace=True)

    # 6️⃣ 항목을 인덱스로
    sub.set_index("항목", inplace=True)
    dfs.append(sub)

# 7️⃣ 병합
df_merged = pd.concat(dfs, axis=1)
df_merged = df_merged.reset_index().sort_values("항목").reset_index(drop=True)

# 8️⃣ ✅ 컬럼명 정제: 언더바 뒤쪽 날짜만 남기기 (혹시 `_`가 여러 개여도 안전하게)
def clean_col(col):
    # 만약 컬럼명이 튜플일 경우 문자열로 합침
    if isinstance(col, tuple):
        col = "_".join([str(c) for c in col if str(c) != 'nan'])
    col = str(col)
    # 언더바가 있고, 언더바 뒤가 날짜 패턴이면 뒤만 추출
    if "_" in col and re.search(r"\d{4}\.\d{2}\.\d{2}", col):
        return re.findall(r"(\d{4}\.\d{2}\.\d{2})", col)[-1]
    else:
        return col.strip()

df_merged.columns = [clean_col(c) for c in df_merged.columns]

# 9️⃣ 엑셀로 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date2.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 컬럼명 정리 완료! → {save_path}")



✅ 컬럼명 정리 완료! → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date2.xlsx


In [24]:
import pandas as pd

file1 = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date1.xlsx"
file2 = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date2.xlsx"

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# ✅ (1) '항목' 컬럼에서 괄호와 그 뒤 내용 제거
df1['항목'] = df1['항목'].astype(str).str.replace(r"\(.*?\)", "", regex=True).str.strip()
df2['항목'] = df2['항목'].astype(str).str.replace(r"\(.*?\)", "", regex=True).str.strip()

# ✅ (2) 중복된 항목명 있을 경우 첫 번째만 남기기 (선택)
df1 = df1.drop_duplicates(subset='항목', keep='first')
df2 = df2.drop_duplicates(subset='항목', keep='first')

# ✅ (3) '항목' 기준으로 outer 병합
df_merged = pd.merge(df1, df2, on='항목', how='outer')

# ✅ (4) 항목 정렬
df_merged = df_merged.sort_values('항목').reset_index(drop=True)

# ✅ (5) 엑셀로 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 병합 완료! 괄호 제거 및 항목 기준 병합 완료 → {save_path}")



✅ 병합 완료! 괄호 제거 및 항목 기준 병합 완료 → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date.xlsx


In [25]:
## 일단시각화 먼저..!
##  영업이익률 = 영업이익/매출액 * 100

In [26]:
DATA_FILE = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\date2.xlsx"
df = pd.read_excel(DATA_FILE)


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   항목          42 non-null     object 
 1   2023.11.14  20 non-null     float64
 2   2024.05.16  21 non-null     float64
 3   2024.11.14  21 non-null     float64
 4   2025.05.15  21 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.8+ KB


In [28]:
df['항목'] = (
    df['항목']
    .astype(str)
    .str.replace(r'\(.*?\)', '', regex=True)  # ✅ 괄호 안 어떤 내용이든 제거
    .str.replace(r'\d+$', '', regex=True)     # ✅ 끝 숫자 제거
    .str.strip()                              # ✅ 공백 제거
)

# 항목 기준으로 같은 이름끼리 합치기
df = df.groupby('항목', as_index=False).sum()



In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   항목          38 non-null     object 
 1   2023.11.14  38 non-null     float64
 2   2024.05.16  38 non-null     float64
 3   2024.11.14  38 non-null     float64
 4   2025.05.15  38 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.6+ KB


In [30]:
df

,항목,2023.11.14,2024.05.16,2024.11.14,2025.05.15
0,금융원가,1.276367e+10,1.491882e+10,1.719956e+10,1.757767e+10
1,기본주당이익,2.424000e+03,7.000000e+01,5.040000e+02,1.465000e+03
2,기타금융손익,0.000000e+00,0.000000e+00,-3.857610e+08,0.000000e+00
3,기타금융수익,5.567541e+08,4.676718e+09,0.000000e+00,3.574659e+09
4,기타비용,0.000000e+00,7.409749e+09,1.027440e+10,8.205650e+09
5,기타손실,3.615236e+09,0.000000e+00,0.000000e+00,0.000000e+00
6,기타수익,0.000000e+00,3.133595e+10,8.455952e+10,8.921137e+10
7,기타이익,1.455330e+11,0.000000e+00,0.000000e+00,0.000000e+00
8,기타포괄손익,7.119210e+08,0.000000e+00,0.000000e+00,0.000000e+00
9,기타포괄손익-공정가치금융상품평가손익,0.000000e+00,0.000000e+00,0.000000e+00,2.845300e+06
